In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    setting = yaml.load(yaml_file)

path = make_path_dict(setting)

In [ ]:
if setting["signal_type"] == "raw":

    raise

In [ ]:
feature_context = pd.read_table(
    path["feature_x_sample.feature_context.tsv"], index_col=0
)

In [ ]:
sample_context = pd.read_table(
    path["feature_x_sample.sample_context.tsv"], index_col=0
).T

In [ ]:
feature_context_negative = -feature_context.clip(upper=0)

feature_context_positive = feature_context.clip(lower=0)

In [ ]:
sample_context_negative = -sample_context.clip(upper=0)

sample_context_positive = sample_context.clip(lower=0)

In [ ]:
if "feature" in setting["element_types"] and "sample" in setting["element_types"]:

    context_negative = ccal.normalize_df(
        feature_context_negative,
        None,
        setting["context_signal_normalization_method"],
        raise_for_bad=False,
    ) + ccal.normalize_df(
        sample_context_negative,
        None,
        setting["context_signal_normalization_method"],
        raise_for_bad=False,
    )

    context_positive = ccal.normalize_df(
        feature_context_positive,
        None,
        setting["context_signal_normalization_method"],
        raise_for_bad=False,
    ) + ccal.normalize_df(
        sample_context_positive,
        None,
        setting["context_signal_normalization_method"],
        raise_for_bad=False,
    )

elif "feature" in setting["element_types"]:

    context_negative = feature_context_negative

    context_positive = feature_context_positive

elif "sample" in setting["element_types"]:

    context_negative = sample_context_negative

    context_positive = sample_context_positive

In [ ]:
title_template = "{} Ranking by Context {} Signal"

value_name_template = "{} Signal Sum"

In [ ]:
if setting["select_feature_automatically"]:

    context_negative_features = ccal.select_series_low_and_high_index(
        context_negative.sum(axis=1),
        title=title_template.format(setting["feature_alias"], "Negative"),
        value_name=value_name_template.format("Negative"),
    )[1]

    context_positive_features = ccal.select_series_low_and_high_index(
        context_positive.sum(axis=1),
        title=title_template.format(setting["feature_alias"], "Positive"),
        value_name=value_name_template.format("Positive"),
    )[1]

else:

    context_negative_features = (
        context_negative.replace(0, np.nan).dropna(how="all").index
    )

    context_positive_features = (
        context_positive.replace(0, np.nan).dropna(how="all").index
    )

context_negative_features.size, context_positive_features.size

In [ ]:
if setting["select_sample_automatically"]:

    title_template = "{} Ranking by Context {{}} Signal".format(setting["sample_alias"])

    value_name_template = "{} Signal Sum"

    context_negative_samples = ccal.select_series_low_and_high_index(
        context_negative.sum(),
        title=title_template.format(setting["sample_alias"], "Negative"),
        value_name=value_name_template.format("Negative"),
    )[1]

    context_positive_samples = ccal.select_series_low_and_high_index(
        context_positive.sum(),
        title=title_template.format(setting["sample_alias"], "Positive"),
        value_name=value_name_template.format("Positive"),
    )[1]

else:

    context_negative_samples = (
        context_negative.replace(0, np.nan).dropna(how="all", axis=1).columns
    )

    context_positive_samples = (
        context_positive.replace(0, np.nan).dropna(how="all", axis=1).columns
    )

context_negative_samples.size, context_positive_samples.size

In [ ]:
context_negative_signal = context_negative.loc[
    context_negative_features, context_negative_samples
]

In [ ]:
context_positive_signal = context_positive.loc[
    context_positive_features, context_positive_samples
]

In [ ]:
if "negative" in setting["contexts"] and "positive" in setting["contexts"]:

    context_negative_signal.index = (
        "(-) {}".format(feature) for feature in context_negative_signal.index
    )

    context_positive_signal.index = (
        "(+) {}".format(feature) for feature in context_positive_signal.index
    )

    context_signal = pd.concat((context_negative_signal, context_positive_signal))

elif "negative" in setting["contexts"]:

    context_signal = context_negative_signal

elif "positive" in setting["contexts"]:

    context_signal = context_positive_signal

In [ ]:
if setting["context_signal_normalization_method"] is not None:

    context_signal = ccal.normalize_df(
        context_signal,
        setting["context_signal_normalization_axis"],
        setting["context_signal_normalization_method"],
        raise_for_bad=False,
    )

In [ ]:
context_signal.to_csv(path["feature_x_sample.signal.tsv"], sep="\t")

ccal.summarize_feature_x_sample(
    context_signal,
    feature_x_sample_alias="Context Signal",
    feature_alias=setting["feature_alias"],
    sample_alias=setting["sample_alias"],
    feature_x_sample_value_name="Context Signal",
    plot_heat_map_max_size=setting["plot_heat_map_max_size"],
    plot_distributions_max_size=setting["plot_distributions_max_size"],
    plot_rug_max_size=setting["plot_rug_max_size"],
)